# Работа с базами данных в python
[Руководство и ссылки](https://habr.com/ru/post/321510/)

[6 нормальных форм СУБД](https://habr.com/ru/post/254773/#:~:text=%D0%A8%D0%B5%D1%81%D1%82%D1%8C%20%D0%BD%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D1%85%20%D1%84%D0%BE%D1%80%D0%BC,-MySQL%2C&text=%D0%92%20%D0%B4%D0%B0%D0%BD%D0%BD%D0%BE%D0%B9%20%D1%82%D0%B5%D0%BC%D0%B5%20%D1%8F%20%D0%B7%D0%B0%D1%82%D1%80%D0%BE%D0%BD%D1%83,%D0%B2%D1%8B%D1%81%D0%BE%D0%BA%D0%BE%D0%B3%D0%BE%20%D0%BF%D0%BE%D1%80%D1%8F%D0%B4%D0%BA%D0%B0%20%D0%BF%D0%BE%20%D0%BE%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%BD%D1%8B%D0%BC%20%D0%BF%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0%D0%BC.)

[Краткое руководство по проектированию СУБД в 15 частях](https://habr.com/ru/post/193136/)

![DB Scheme](db_scheme.png)

В данной лкции будет использоваться открытая (MIT лицензия) тестовая база данных “Chinook”. Скачать ее можно с [репозитория](github.com/lerocha/chinook-database)

Нужен для работы только бинарный файл [Chinook_Sqlite.sqlite](github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite):


Для удобства работы с базой (просмотр, редактирование) нам нужна программа браузер баз данных, поддерживающая SQLite, например, [DB Browser](https://sqlitebrowser.org/) 

In [1]:
# Импортируем библиотеку, соответствующую типу нашей базы данных 
import sqlite3

# Создаем соединение с нашей базой данных
# В нашем примере у нас это просто файл базы
conn = sqlite3.connect('Chinook_Sqlite.sqlite')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

# Не забываем закрыть соединение с базой данных
conn.close()

In [2]:
# Создаем соединение с нашей базой данных
# В нашем примере у нас это просто файл базы
conn = sqlite3.connect('Chinook_Sqlite.sqlite')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

In [3]:
# Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")

# Получаем результат сделанного запроса
results = cursor.fetchall()
results2 =  cursor.fetchall()

print(results)   # [('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',)]
print(results2)  # []

[('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',)]
[]


In [4]:
# Делаем INSERT запрос к базе данных, используя обычный SQL-синтаксис
cursor.execute("insert into Artist values (Null, 'A Aagrh!') ")

# Если мы не просто читаем, но и вносим изменения в базу данных - необходимо сохранить транзакцию
conn.commit()

# Проверяем результат
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")
results = cursor.fetchall()
print(results)  # [('A Aagrh!',), ('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',)]

[('A Aagrh!',), ('A Cor Do Som',), ('AC/DC',)]


In [5]:
cursor.execute("""
  SELECT Name
  FROM Artist
  ORDER BY Name LIMIT 3
""")
results = cursor.fetchall()
print(results)

[('A Aagrh!',), ('A Cor Do Som',), ('AC/DC',)]


In [6]:
def print_last(num = 3):
    cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT " + str(num))
    print(cursor.fetchall())


In [11]:
#cursor.execute("""insert into Artist values (Null, 'A Aagrh!');""")
#cursor.execute("""insert into Artist values (Null, 'A Aagrh-2!');""")

cursor.executescript("""
 insert into Artist values (Null, 'A Aagrh!');
 insert into Artist values (Null, 'A Aagrh-2!');
""")
print_last(10)

[('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh-2!',), ('A Aagrh-2!',), ('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',)]


In [13]:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT ?", ('2'))
print(cursor.fetchall())

# И с использованием именнованных замен:
cursor.execute("SELECT Name from Artist ORDER BY Name LIMIT :limit", {"limit": 3})
# В PostgreSQL (UPD: и в MySQL) вместо знака '?' для подстановки используется: %s
print(cursor.fetchall())


[('A Aagrh!',), ('A Aagrh!',)]
[('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',)]


In [14]:
# Обратите внимание, даже передавая одно значение - его нужно передавать кортежем!
# Именно по этому тут используется запятая в скобках!
new_artists = [
    ('A Aagrh!',),
    ('A Aagrh!-2',),
    ('A Aagrh!-3',),
]
cursor.executemany("insert into Artist values (Null, ?);", new_artists)

print_last(10)

[('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!-2',), ('A Aagrh!-3',), ('A Aagrh-2!',), ('A Aagrh-2!',), ('A Cor Do Som',)]


In [15]:
print_last(30)

[('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!-2',), ('A Aagrh!-3',), ('A Aagrh-2!',), ('A Aagrh-2!',), ('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',), ('Academy of St. Martin in the Fields & Sir Neville Marriner',), ('Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner',), ('Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair',), ('Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart',), ('Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett',), ('Accept',), ('Adrian Leaper & Doreen de Feis',), ('Aerosmith',), ("Aerosmith & Sierra Leone's Refugee Allstars",), ('Aisha Duo',), ('Alanis Morissette',), ('Alberto Turco & Nova Schola Gregoriana',), ('Alice In Chains',), ('Amy Winehouse',), ('Anne-Sophie Mutter, Herbert Von Karajan & Wiener Philharmoniker',), ('Antal Doráti & London Symphony Orchestra

In [32]:
cursor.execute("SELECT * FROM Artist ORDER BY Name LIMIT 30")
row = cursor.fetchone()
while not row == None:
    print(row[0], row[1])
    
    row = cursor.fetchone()
#print(cursor.fetchone())    # ('A Cor Do Som',)
#print(cursor.fetchone())    # ('Aaron Copland & London Symphony Orchestra',)
#print(cursor.fetchone())    # ('Aaron Goldberg',)
#print(cursor.fetchone())    # None

276 A Aagrh!
277 A Aagrh!
278 A Aagrh!
280 A Aagrh!
282 A Aagrh!
283 A Aagrh!-2
284 A Aagrh!-3
279 A Aagrh-2!
281 A Aagrh-2!
43 A Cor Do Som
1 AC/DC
230 Aaron Copland & London Symphony Orchestra
202 Aaron Goldberg
214 Academy of St. Martin in the Fields & Sir Neville Marriner
215 Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner
222 Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair
257 Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart
239 Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett
2 Accept
260 Adrian Leaper & Doreen de Feis
3 Aerosmith
161 Aerosmith & Sierra Leone's Refugee Allstars
197 Aisha Duo
4 Alanis Morissette
206 Alberto Turco & Nova Schola Gregoriana
5 Alice In Chains
252 Amy Winehouse
209 Anne-Sophie Mutter, Herbert Von Karajan & Wiener Philharmoniker
243 Antal Doráti & London Symphony Orchestra
6 Antônio Carlos Jobim


In [19]:
for row in cursor.execute('SELECT Name from Artist ORDER BY Name LIMIT 3'):
    print(row)

('A Aagrh!',)
('A Aagrh!',)
('A Aagrh!',)


In [22]:
sql_statement = "select * from Artist where name like 'A Aagrh!%'"
try:
    cursor.execute(sql_statement)
    result = cursor.fetchall()
    print(result)
except sqlite3.DatabaseError as err:       
    print("Error: ", err)
else:
    conn.commit()

[(276, 'A Aagrh!'), (277, 'A Aagrh!'), (278, 'A Aagrh!'), (280, 'A Aagrh!'), (282, 'A Aagrh!'), (283, 'A Aagrh!-2'), (284, 'A Aagrh!-3')]


In [ ]:
# для postgres
# with psycopg2.connect("dbname='habr'") as conn:
#   with conn.cursor() as cur:

In [23]:
# В самом конце 
conn.close()

In [35]:
import sqlite3
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

con = sqlite3.connect("Chinook_Sqlite.sqlite")
#cursor = con.cursor()
con.row_factory = dict_factory
cur = con.cursor()
cur.execute("select * from Artist where Name like 'A Aagrh!%'")
for row in cur.fetchall():
    print(row["Name"], row["ArtistId"])
con.close()

A Aagrh! 276
A Aagrh! 277
A Aagrh! 278
A Aagrh! 280
A Aagrh! 282
A Aagrh!-2 283
A Aagrh!-3 284


In [38]:
import sqlite3

conn = sqlite3.connect("Chinook_Sqlite.sqlite")
cursor = conn.cursor()

try:
    sql_statement = "delete from Artist where name like 'A Aagrh%'"
    cursor.execute(sql_statement)
    result = cursor.fetchall()
    print(result)
    print_last(30)
except sqlite3.DatabaseError as err:       
    print("Error: ", err)
else:
    conn.commit()
    conn.close()

[]
[('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',), ('Academy of St. Martin in the Fields & Sir Neville Marriner',), ('Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner',), ('Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair',), ('Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart',), ('Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett',), ('Accept',), ('Adrian Leaper & Doreen de Feis',), ('Aerosmith',), ("Aerosmith & Sierra Leone's Refugee Allstars",), ('Aisha Duo',), ('Alanis Morissette',), ('Alberto Turco & Nova Schola Gregoriana',), ('Alice In Chains',), ('Amy Winehouse',), ('Anne-Sophie Mutter, Herbert Von Karajan & Wiener Philharmoniker',), ('Antal Doráti & London Symphony Orchestra',), ('Antônio Carlos Jobim',), ('Apocalyptica',), ('Aquaman',), ('Audioslave',), ('Avril Lavigne',), ('Azymuth',), ('Baby Consuelo',), ('Ba

# DB ORM
![ORM](orm_scheme.jpeg)


[ORM python](https://habr.com/ru/post/322086/)


Существуют два основных подхода к реализации ORM:
Active Record – более простой для понимания и реализации подход, суть которого сводится к отображению объекта данных на строку базы данных.
Data Mapper – в отличии от предыдущего подхода полностью разделяет представление данных в программе от его представления в базе данных.

In [39]:
# Импортируем библиотеку, соответствующую типу нашей базы данных 
# В данном случае импортируем все ее содержимое, чтобы при обращении не писать каждый раз имя библиотеки, как мы делали в первой статье
from peewee import *

# Создаем соединение с нашей базой данных
# В нашем примере у нас это просто файл базы
conn = SqliteDatabase('Chinook_Sqlite.sqlite')

# ТУТ БУДЕТ КОД НАШИХ МОДЕЛЕЙ

# Создаем курсор - специальный объект для запросов и получения данных с базы
cursor = conn.cursor()

# Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")

# Получаем результат сделанного запроса
results = cursor.fetchall()
print(results)

# Не забываем закрыть соединение с базой данных
conn.close()

[('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',)]


True

In [40]:
# Импортируем нашу ORM библиотеку
from peewee import *

# Создаем соединение с нашей базой данных
conn = SqliteDatabase('Chinook_Sqlite.sqlite')




In [41]:
################ 3, ОПРЕДЕЛЯЕМ МОДЕЛИ ######################

# Определяем базовую модель о которой будут наследоваться остальные
class BaseModel(Model):
    class Meta:
        database = conn


# Определяем модель исполнителя
class Artist(BaseModel):
    artist_id = AutoField(column_name='ArtistId')
    name = TextField(column_name='Name', null=True)

    class Meta:
        table_name = 'Artist'


def print_last_five_artists():
    """ Печатаем последние 5 записей в таблице исполнителей"""
    print('########################################################')
    cur_query = Artist.select().limit(5).order_by(Artist.artist_id.desc())
    for item in cur_query.dicts().execute():
        print('artist: ', item)


# Создаем курсор - это специальный объект который делает запросы
# и получает их результаты
cursor = conn.cursor()



In [42]:
################ 2, ИСПОЛЬЗУЕМ КУРСОР  ###################

# Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")

# Получаем результат сделанного запроса
results = cursor.fetchall()
print(results)   # [('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',)]




[('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',)]


In [43]:
######################## 5, ЧИТАЕМ ИЗ БАЗЫ ########################

# 5.1 Получение одиночной записи с методом модели Model.get()
artist = Artist.get(Artist.artist_id == 1)
# теперь у нас есть объект artist,
# с полями соответствующим данным исполнителя в конкретной строке
# а также доступными методами модели исполнителя
# этот объект можно использовать не только для чтения данных,
# но и для их обновления и удаления данной записи, в чем убедимся позже
print('artist: ', artist.artist_id, artist.name)  # artist:  1 AC/DC

# 5.2 Получение набора записей похоже на стандартный select запрос к базе,
# но осуществляемый через нашу модель Model.select()
# Обратите внимание, что к какой таблице обращаться и какие поля у нее есть
# уже определено в нашей модели и нам не надо это указывать в нашем запросе

# Формируем запрос к базе с помощью нашей ORM прослойки
# и смотрим как этот запрос будет выглядеть
query = Artist.select()
print(query)
# SELECT "t1"."ArtistId", "t1"."Name" FROM "Artist" AS "t1"

# Полезно добавить дополнительные параметры,
# уточняющие запрос, они очень похожи на SQL инструкции:
query = Artist.select().where(Artist.artist_id < 10).\
                        limit(5).order_by(Artist.artist_id.desc())
print(query)
# SELECT "t1"."ArtistId", "t1"."Name" FROM "Artist" AS "t1"
#   WHERE ("t1"."ArtistId" < 10) ORDER BY "t1"."ArtistId" DESC LIMIT 5

# Теперь, определившись с запросом к базе, мы можем получить от нее ответ,
# для удобства делаем это сразу в виде словаря
artists_selected = query.dicts().execute()
print(artists_selected)
# <peewee.ModelDictCursorWrapper object at 0x7f6fdd9bdda0>
# это итератор по полученным из базы записям, который можно обходить в цикле
for artist in artists_selected:
    print('artist: ', artist)   # artist:  {'artist_id': 9, 'name': 'BackBeat'}
    # То есть, каждая итерация соответствует одной строке таблицы
    # и соответственно одному исполнителю




artist:  1 AC/DC
SELECT "t1"."ArtistId", "t1"."Name" FROM "Artist" AS "t1"
SELECT "t1"."ArtistId", "t1"."Name" FROM "Artist" AS "t1" WHERE ("t1"."ArtistId" < 10) ORDER BY "t1"."ArtistId" DESC LIMIT 5
artist:  {'artist_id': 9, 'name': 'BackBeat'}
artist:  {'artist_id': 8, 'name': 'Audioslave'}
artist:  {'artist_id': 7, 'name': 'Apocalyptica'}
artist:  {'artist_id': 6, 'name': 'Antônio Carlos Jobim'}
artist:  {'artist_id': 5, 'name': 'Alice In Chains'}


In [44]:
################  6, СОЗДАЕМ НОВУЮ ЗАПИСЬ В БАЗЕ  #######################

# 6.1 Первый способ: Model.create() - передаем все требуемые параметры сразу
Artist.create(name='1-Qwerty')

# 6.2 Второй способ: Мы создаем объект класса нашей модели,
# работаем в коде в содержимым его полей,
# а в конце вызываем его метод .save()
artist = Artist(name='2-asdfg')
artist.save()  # save() returns the number of rows modified.
# обратите внимание, что здесь метод вызываем у объекта класса модели,
# а не у самой модели, как в первом способе

# 6.3 Третий способ - массовое добавление из коллекции
# методом модели Model.insert_many()
# Обратите внимание, что первые два метода не требуют добавления .execute(),
# а этот требует!
artists_data = [{'name': '3-qaswed'}, {'name': '4-yhnbgt'}]
Artist.insert_many(artists_data).execute()

# Визуализируем последние 5 записей в таблице исполнителей,
# чтобы убедится. что к последней добавлены 4 новые
print_last_five_artists()
# artist:  {'artist_id': 279, 'name': '4-yhnbgt'}
# artist:  {'artist_id': 278, 'name': '3-qaswed'}
# artist:  {'artist_id': 277, 'name': '2-asdfg'}
# artist:  {'artist_id': 276, 'name': '1-Qwerty'}
# artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}




########################################################
artist:  {'artist_id': 279, 'name': '4-yhnbgt'}
artist:  {'artist_id': 278, 'name': '3-qaswed'}
artist:  {'artist_id': 277, 'name': '2-asdfg'}
artist:  {'artist_id': 276, 'name': '1-Qwerty'}
artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}


In [45]:
############### 7, ОБНОВЛЯЕМ ДАННЫЕ СУЩЕСТВУЮЩЕЙ ЗАПИСИ ##############

# 7.1 Выше, способом 6.2 мы создавали новую запись,
# но так можно не только создавать новую запись, но и обновлять существующую.
# Для этого нам надо для нашего объекта указать
# уже существующий в таблице первичный ключ.

artist = Artist(name='2-asdfg+++++')
artist.artist_id = 277  # Тот самый первичный ключ
# который связывает наш объект с конкретной строке таблицы базы данных
artist.save()

print_last_five_artists()
# artist:  {'artist_id': 279, 'name': '4-yhnbgt'}
# artist:  {'artist_id': 278, 'name': '3-qaswed'}
# artist:  {'artist_id': 277, 'name': '2-asdfg+++++'}
# artist:  {'artist_id': 276, 'name': '1-Qwerty'}
# artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}

# 7.2 Для обновления многих записей сразу,
# можно использовать метод модели Model.update(),
# в котором указываем что именно у нас меняется,
# а метод .where() определяет по каким критериям отбираются записи
query = Artist.update(name=Artist.name + '!!!').where(Artist.artist_id > 275)
query.execute()

print_last_five_artists()
# artist:  {'artist_id': 279, 'name': '4-yhnbgt!!!'}
# artist:  {'artist_id': 278, 'name': '3-qaswed!!!'}
# artist:  {'artist_id': 277, 'name': '2-asdfg+++!!!'}
# artist:  {'artist_id': 276, 'name': '1-Qwerty!!!'}
# artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}




########################################################
artist:  {'artist_id': 279, 'name': '4-yhnbgt'}
artist:  {'artist_id': 278, 'name': '3-qaswed'}
artist:  {'artist_id': 277, 'name': '2-asdfg+++++'}
artist:  {'artist_id': 276, 'name': '1-Qwerty'}
artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}
########################################################
artist:  {'artist_id': 279, 'name': '4-yhnbgt!!!'}
artist:  {'artist_id': 278, 'name': '3-qaswed!!!'}
artist:  {'artist_id': 277, 'name': '2-asdfg+++++!!!'}
artist:  {'artist_id': 276, 'name': '1-Qwerty!!!'}
artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}


In [52]:
###################### 8. УДАЛЯЕМ ЗАПИСЬ #######################

# 8.1 Первый способ удаления записи -
# это получение объекта записи методом Model.get() как в 5.1 выше
artist = Artist.get(Artist.artist_id == 257)
# и вызова метода удаления этой записи .delete_instance():
artist.delete_instance()

print_last_five_artists()
# artist:  {'artist_id': 278, 'name': '3-qaswed!!!'}
# artist:  {'artist_id': 277, 'name': '2-asdfg+++!!!'}
# artist:  {'artist_id': 276, 'name': '1-Qwerty!!!'}
# artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}
# artist:  {'artist_id': 274, 'name': 'Nash Ensemble'}

# 8.2 Для удаления набора строк можно использовать Model.delete() метод
query = Artist.delete().where(Artist.artist_id > 275)
query.execute()

print_last_five_artists()
# artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}
# artist:  {'artist_id': 274, 'name': 'Nash Ensemble'}
# artist:  {'artist_id': 273, 'name': 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu'}
# artist:  {'artist_id': 272, 'name': 'Emerson String Quartet'}
# artist:  {'artist_id': 271, 'name': 'Mela Tenenbaum, Pro Musica Prague & Richard Kapp'}


# Не забываем закрыть соединение с базой данных в конце работы
if conn.close():
    print('Connection closed')

########################################################
artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}
artist:  {'artist_id': 274, 'name': 'Nash Ensemble'}
artist:  {'artist_id': 273, 'name': 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu'}
artist:  {'artist_id': 272, 'name': 'Emerson String Quartet'}
artist:  {'artist_id': 271, 'name': 'Mela Tenenbaum, Pro Musica Prague & Richard Kapp'}
########################################################
artist:  {'artist_id': 275, 'name': 'Philip Glass Ensemble'}
artist:  {'artist_id': 274, 'name': 'Nash Ensemble'}
artist:  {'artist_id': 273, 'name': 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu'}
artist:  {'artist_id': 272, 'name': 'Emerson String Quartet'}
artist:  {'artist_id': 271, 'name': 'Mela Tenenbaum, Pro Musica Prague & Richard Kapp'}
Connection closed


In [ ]:
#  python -m pwiz -e sqlite ../Chinook_Sqlite.sqlite  >> models.py